# Data Preparation Fresh 68k PBMCs (Donor A)

This notebook prepares the 68K PBMC dataset for further analysis. 

In [1]:
# loading packages
from scipy.io import mmread
import pandas as pd
import anndata
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
import scanpy as sc

## Data preprocessing

This code defines paths and loads the 68k PBMC dataset with annotations. It transposes the DataFrame, because the input data for Geneformer should have the cells located in the columns. After transposing, the index column contains the column name as a value, which is why the index should be resetted.

In [2]:
# define paths
file_path_sparse = "/home/cog/nlandzaat/rep_learning_sc_foundation/raw/PBMC/sparse_matrix_pmbc68kdata.txt"

# load dataset
with open(file_path_sparse) as matrix_file:
    matrix_sparse = mmread(matrix_file)

df = pd.DataFrame.sparse.from_spmatrix(matrix_sparse)

# load annotations
gene_ID_names = pd.read_csv("/home/cog/nlandzaat/rep_learning_sc_foundation/raw/PBMC/annotatie.csv")
df.columns = gene_ID_names["ENSEMBL_IDS"]

In [3]:
# transpose DataFrame
df_transposed = df.T

# reset the index
df_transposed["ENSEMBL_IDS"] = df_transposed.index
df_transposed = df_transposed.reset_index(drop=True)

# move "ENSEMBL_IDS" column to the first position
df_transposed = df_transposed[["ENSEMBL_IDS"] + [col for col in df_transposed.columns if col != "ENSEMBL_IDS"]]

/scratch/17309981/ipykernel_2496279/977535884.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transposed["ENSEMBL_IDS"] = df_transposed.index


Map de gene names to the Ensembl IDs, this is necessary because the gene names need to be specified in order to use the Geneformer model. Not all Ensembl IDs are recognized, resulting in NaN in the Gene_names column. These genes are dropped from the dataset. The missing Ensembl IDs are listed in missing_ids_list.

In [4]:
# load the file with Ensembl ID to gene name mapping
mapping_df = pd.read_csv("/home/cog/nlandzaat/rep_learning_sc_foundation/raw/PBMC/mart_export.txt", delimiter=",")

# create a mapping dictionary from "Gene stable ID" to "Gene name"
mapping_dict = mapping_df.set_index("Gene stable ID")["Gene name"].to_dict()

# map the gene names to df_transposed based on the ensemble IDs
df_transposed["Gene_names"] = df_transposed["ENSEMBL_IDS"].map(mapping_dict)

# these Ensembl IDs do not have a matching gene name
missing_ids = df_transposed[~df_transposed["ENSEMBL_IDS"].isin(mapping_df["Gene stable ID"])]["ENSEMBL_IDS"]
missing_ids_list = missing_ids.to_list()

# remove rows (genes) with NaN
df_cleaned = df_transposed.dropna()

/scratch/17309981/ipykernel_2496279/2608749983.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transposed["Gene_names"] = df_transposed["ENSEMBL_IDS"].map(mapping_dict)


### Prepare data for foundation models

Geneformer requires gene names, we dropped the Ensembl IDs and only kept the gene names. Dropping the Ensembl IDs is not necessary to leave them out of the models' calculation, since the models ask for specification of the layer where the raw counts are stored.

The following code creates an AnnData object (adata_GF) from the DataFrame (df_gf) containing gene expression data. The DataFrame is transposed to have samples as rows and genes as columns, aligning with the format expected by the AnnData object. The AnnData object is initialized with the transposed expression data (X), where each row represents a sample (cell) and each column represents a gene. Additionally, empty DataFrame objects are created for observations (obs) and variables (var). The code then generates cell names for each sample in the observations DataFrame, using the format "cell 1", "cell 2", and so on. Additionally, the raw counts are stored in the layer "counts", which is necessary for extracting the raw counts by the foundation models.

In [5]:
# drop Ensembl IDs
df_gf = df_cleaned.drop("ENSEMBL_IDS", axis=1)
df_gf.set_index("Gene_names", inplace=True)
df_gf.index.name = None


In [10]:
# create an AnnData object 
adata_GF = anndata.AnnData(X=df_gf.values.T, var=pd.DataFrame(index=df_gf.index), obs=pd.DataFrame(index=df_gf.columns))

# create cell names in obs
adata_GF.obs["cell_names"] = ["PBMC" for i in range (len(adata_GF.obs))]

/hpc/compgen/users/nlandzaat/Software/anaconda3/envs/project_env_test/lib/python3.9/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [12]:
# create a layer named "counts" and store the raw counts in it
adata_GF.layers["counts"] = df_gf.values.T

The AnnData object is prepared for use by the foundation model, it is stored to an h5ad file so that it can be accessed later. Before converting the AnnData file, modifications are necessary. The indeces of the variables and observations attributes are converted to strings. The h5ad file is saved to the specified file path.

In [14]:
# modify and store the AnnData object
file_path = "/home/cog/nlandzaat/rep_learning_sc_foundation/raw/PBMC/adata_GF.h5ad"

adata_GF.var.index = adata_GF.var.index.astype(str)
adata_GF.obs.index = adata_GF.obs.index.astype(str)
# save AnnData to h5ad file
adata_GF.write_h5ad(file_path)

: 

### Prepare data for PCA and UMAP

We start with a DataFrame (df_cleaned) where genes are represented as rows and samples as columns. To align with the requirements of PCA and UMAP, we transpose the DataFrame to have samples as rows and genes as columns. We then drop the "Gene_names" column and move the Ensembl IDs to the index, in order to exclude them from calculations. Next, we convert the DataFrame to numeric values using pd.to_numeric to ensure consistency and compatibility with PCA and UMAP. As the dataset is large, we convert it to a sparse matrix for memory efficiency. Finally, we create an AnnData object using the sparse matrix.

In [35]:
# move ensemble ids to index 
df_cleaned = df_cleaned.set_index(df_cleaned.columns[0])

# transpose data frame 
df_pca = df_cleaned.T

In [36]:
# drop Gene_names column
row_to_drop = ["Gene_names"]
df_pca = df_pca.drop(row_to_drop, axis=0)

# convert to numeric
df_pca = df_pca.apply(pd.to_numeric, errors="coerce")

In [38]:
# convert the entire DataFrame to a sparse matrix
sparse_matrix = csr_matrix(df_pca.values)

# convert the DataFrame to an AnnData object 
adata = sc.AnnData(X=sparse_matrix)

In [36]:
# save adata

# specify the file path 
adata_file_path = "/home/cog/nlandzaat/rep_learning_sc_foundation/raw/PBMC/adata_PBMC_PCA_UMAP.h5ad"

# write adata to the specified file path
adata.write(adata_file_path)